# Regional Level Alerts


> 👋 Before moving on with this demo, you must first sign-up and request your Geosys APIs credentials here :
> - ⚙️[Try it now](https://www.earthdaily.com/geosys/geosys-api/)

> For more information about our Geosys APIs : 
> - 📚 [Geosys APIs to connect with your digital ag application](https://www.earthdaily.com/geosys/geosys-api/)


> **Demo Project:** This demo demonstrates the ability to get weather or vegetation alerts on a regional entity based on a defined parameter (weather or vegetation) and threshold.



### @author: Geosys



 ## 1️⃣ Import all librairies needed and get an autorization to use RegionalLevelAlerts

In [ ]:
import sys
import os
sys.path.append(os.path.join('..','src'))

import logging
from dotenv import load_dotenv
from datetime import datetime
from geosyspy import Geosys
from geosyspy.utils.constants import *
from regionallevelalerts.processor import RegionalLevelAlerts
from regionallevelalerts import utils

logger = logging.getLogger()
logger.setLevel(logging.WARN)

# read .env file
load_dotenv()

API_CLIENT_ID = os.getenv('API_CLIENT_ID')
API_CLIENT_SECRET = os.getenv('API_CLIENT_SECRET')
API_USERNAME = os.getenv('API_USERNAME')
API_PASSWORD = os.getenv('API_PASSWORD')

# Create and initialize the client
print("Initializing the client...")
client = RegionalLevelAlerts(API_CLIENT_ID, API_CLIENT_SECRET, API_USERNAME, API_PASSWORD, Env.PROD, Region.NA)
print("Client initialized !")


 ## 2️⃣ Get Weather Alerts

The following parameters are used as an input to get regional entities in alert:

##### blockCode: 
The Block containing the regional entities to analyze

##### weatherType: 
The type of weather to take into account to get alerts

#### operator: 
The operator used to compare weather values and the threshold

#### startDate: 
A datetime object representing the beginning of the period to analyze

#### endDate: 
A datetime object representing the end of the period to analyze

#### threshold: 
A floating-point value representing the threshold used to determine alerts


In [ ]:
try:
    # Input parameters
    blockCode = AgriquestBlocks.FRA_DEPARTEMENTS
    startDate = datetime.strptime("2022-01-01", "%Y-%m-%d").date()
    endDate = datetime.strptime("2022-12-31", "%Y-%m-%d").date()
    weatherType = AgriquestWeatherType.MAX_TEMPERATURE
    threshold = 100
    operator = utils.NumbersComparisonOperator.GREATER_THAN
    
    # Get weather alerts DataFrame
    regional_level_alerts_df = client.get_weather_alerts(blockCode, startDate, endDate, weatherType, threshold, operator)
    if regional_level_alerts_df is None:
        print("get_weather_alerts: No data found")
    else:
        # Save result DataFrame into a csv
        weather_alerts_csv_file_path = utils.save_dataframe_to_temporary_csv(regional_level_alerts_df, "_weather")
        print(f"get_weather_alerts CSV: {weather_alerts_csv_file_path}")

        # Upload the csv on AWS S3 and Azure Blob Storage
        if utils.write_file_to_aws_s3(weather_alerts_csv_file_path):
           print("File uploaded to AWS")
        if utils.write_file_to_azure_blob_storage(weather_alerts_csv_file_path):
           print("File uploaded to Azure")

        # Optionally save it locally where you want just like this :
        # regional_level_alerts_df.to_csv(<my_file_path_and_name>, index=False)

except Exception as exc:
    print(f"Error: {exc}")


 ## 3️⃣ Get Vegetation Alerts

The following parameters are used as an input to get regional entities in alert:

##### blockCode: 
The Block containing the regional entities to analyze

#### operator: 
The operator used to compare NDVI values and the threshold

#### observationDate: 
A datetime object representing the date to analyze

#### threshold: 
A floating-point value representing the threshold used to determine alerts


In [ ]:
try:
    # Input parameters
    blockCode = AgriquestBlocks.FRA_DEPARTEMENTS
    observationDate = datetime.strptime("2022-12-31", "%Y-%m-%d").date()
    threshold = 0.55
    operator = utils.NumbersComparisonOperator.GREATER_THAN
    
    # Get vegetation alerts DataFrame
    regional_level_alerts_df = client.get_vegetation_alerts(blockCode, observationDate, threshold, operator)
    if regional_level_alerts_df is None:
        print("get_vegetation_alerts: No data found")
    else:
        # Save result DataFrame into a csv
        vegetation_alerts_csv_file_path = utils.save_dataframe_to_temporary_csv(regional_level_alerts_df, "_vegetation")
        print(f"get_vegetation_alerts CSV: {vegetation_alerts_csv_file_path}")

        # Upload the csv on AWS S3 and Azure Blob Storage
        if utils.write_file_to_aws_s3(vegetation_alerts_csv_file_path):
           print("File uploaded to AWS")
        if utils.write_file_to_azure_blob_storage(vegetation_alerts_csv_file_path):
           print("File uploaded to Azure")

        # Optionally save it locally where you want just like this :
        # regional_level_alerts_df.to_csv(<my_file_path_and_name>, index=False)

except Exception as exc:
    print(f"Error: {exc}")
